# Separating Particulate and Dissolved Constituents in ACS Data

In this notebook, processing ACS data and separating the particulate and gelbstoff signals using Xarray are demonstrated. This requires a merged dataset that contains information about the filtration state, the location, temperature, and salinity. In this example dataset, a seawater_state value of 0 indicates seawater (particulate + gelbstoff) and a value of 1 indicates that seawater that has been filtered through a 0.2 micron filter (gelbstoff).

You can rerun this example by downloading the subset dataset and device file from [Kaggle](https://www.kaggle.com/datasets/blackia/shimada202405-subset-acs) and modifying the filepath locations in the following cells.

In [1]:
import gsw
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DayLocator, HourLocator
from matplotlib.ticker import MultipleLocator
import numpy as np
import xarray as xr
import scipy
from scipy.stats import linregress

from acspype import ACSDev, ACSTSCor
import acspype.processing as acsproc
import acspype.qaqc as acsqaqc

In [2]:
data_fp = '../dev_tools/test_files/shimada_202405_subset.nc'
dev_fp = '../dev_tools/test_files/ACS-00412_2023-05-10.dev'

acs = xr.open_dataset(data_fp)
dev = ACSDev(dev_fp)

In [3]:
tscor = ACSTSCor().to_xarray()  ## Load the TSCor coefficients as an xarray dataset.

# TSCor coeffs for absorption.
psi_t_a = tscor.psi_t.sel(wavelength=acs.a_wavelength)
psi_s_a = tscor.psi_s_a.sel(wavelength=acs.a_wavelength)

# TSCor coeffs for attenuation.
psi_t_c = tscor.psi_t.sel(wavelength=acs.c_wavelength)
psi_s_c = tscor.psi_s_c.sel(wavelength=acs.c_wavelength)

## Reprocess Data

In [4]:
acs['internal_temperature'] = acsproc.compute_internal_temperature(counts = acs.raw_internal_temperature)

In [5]:
sea_water_pressure = 3 # Assume flowthrough intake is 3 dbars/meters below the average water line.
acs['sea_water_absolute_salinity'] = gsw.SA_from_SP(acs.sea_water_practical_salinity, sea_water_pressure, acs.longitude, acs.latitude)
acs['sea_water_conservative_temperature'] = gsw.CT_from_t(acs.sea_water_absolute_salinity, acs.sea_water_temperature, sea_water_pressure)

In [6]:
acs['a_uncorrected'] = acsproc.compute_uncorrected(signal_counts=acs.a_signal,
                                                   reference_counts=acs.a_reference, 
                                                   path_length = dev.path_length)
acs['c_uncorrected'] = acsproc.compute_uncorrected(signal_counts = acs.c_signal, 
                                                   reference_counts = acs.c_reference, 
                                                   path_length = dev.path_length)

In [7]:
acs['a_m_discontinuity'] = acsproc.compute_measured(uncorrected = acs.a_uncorrected, 
                                                    internal_temperature = acs.internal_temperature, 
                                                    offset = dev.a_offset, 
                                                    func_delta_t = dev.func_a_delta_t)
acs['c_m_discontinuity'] = acsproc.compute_measured(uncorrected = acs.c_uncorrected, 
                                                    internal_temperature = acs.internal_temperature, 
                                                    offset = dev.c_offset, 
                                                    func_delta_t=dev.func_c_delta_t)

In [8]:
discontinuity_index = acsproc.find_discontinuity_index(a_wavelength = acs.a_wavelength, 
                                                       c_wavelength = acs.c_wavelength)
acs['a_m'], acs['a_discontinuity_offset'] = acsproc.discontinuity_correction(measured = acs.a_m_discontinuity,
                                                                             discontinuity_index=discontinuity_index,
                                                                             wavelength_dim='a_wavelength')
acs['c_m'], acs['c_discontinuity_offset'] = acsproc.discontinuity_correction(measured = acs.c_m_discontinuity,
                                                                             discontinuity_index=discontinuity_index,
                                                                             wavelength_dim='c_wavelength')

In [9]:
tcal = dev.tcal  # The reference temperature value in the device file.
acs['a_mts'] = acsproc.ts_correction(measured = acs.a_m, 
                                     temperature = acs.sea_water_conservative_temperature,
                                     salinity = acs.sea_water_absolute_salinity, 
                                     psi_temperature = psi_t_a, 
                                     psi_salinity= psi_s_a, 
                                     tcal = tcal)
acs['c_mts'] = acsproc.ts_correction(measured = acs.c_m, 
                                     temperature = acs.sea_water_conservative_temperature,
                                     salinity = acs.sea_water_absolute_salinity, 
                                     psi_temperature= psi_t_c, 
                                     psi_salinity = psi_s_c, 
                                     tcal = tcal)

In [10]:
acs['a_mts'] = acsproc.zero_shift_correction(mts = acs.a_mts)
acs['c_mts'] = acsproc.zero_shift_correction(mts = acs.c_mts)

In [11]:
acs = acsproc.interpolate_common_wavelengths(ds = acs,
                                             a_wavelength_dim='a_wavelength',
                                             c_wavelength_dim='c_wavelength',
                                             new_wavelength_dim='wavelength',
                                             wavelength_range='infer',
                                             step = 1)

In [12]:
reference_wavelength = 715
acs = acs.sel(wavelength = slice(None, reference_wavelength))
a_715 = acs.a_mts.sel(wavelength = reference_wavelength, method = 'nearest')
c_715 = acs.c_mts.sel(wavelength = reference_wavelength, method = 'nearest')

In [13]:
acs['a_mts_proportional'] = acsproc.proportional_scattering_correction(a_mts = acs.a_mts, 
                                                                       c_mts = acs.c_mts, 
                                                                       reference_a = a_715, 
                                                                       reference_c = c_715) # Proportional Method from Zaneveld et al. 1994
acs['a_mts_proportional'] = acsproc.zero_shift_correction(mts = acs.a_mts_proportional)

## Remove Poor Data

In [14]:
# Elapsed Time Test
acs['flag_elapsed_time'] = acsqaqc.elapsed_time_test(acs.elapsed_time, fail_threshold = 10 * 60 * 1000, suspect_threshold = 15 * 60 * 1000)

# Internal Temperature Test
acs['flag_internal_temperature'] = acsqaqc.internal_temperature_test(internal_temperature=acs.internal_temperature, dev = dev)

# Inf Nan Teset
acs['flag_a_uncorrected_inf_nan'] = acsqaqc.inf_nan_test(uncorrected = acs.a_uncorrected)
acs['flag_c_uncorrected_inf_nan'] = acsqaqc.inf_nan_test(uncorrected = acs.c_uncorrected)


# Gross Range Test
acs['flag_c_mts_gross_range'] = acsqaqc.gross_range_test(mts = acs.c_mts, 
                                                         sensor_min = -0.005, sensor_max = 10,
                                                         op_min = 0, op_max = 8.5)
acs['flag_a_mts_proportional_gross_range'] = acsqaqc.gross_range_test(mts = acs.a_mts_proportional, 
                                                                      sensor_min = -0.005, sensor_max = 10,
                                                                      op_min = 0, op_max = 8.5)
acs['blanket_flag_c_mts_gross_range'] = acsqaqc.blanket_gross_range_test(acs.flag_c_mts_gross_range, 
                                                                 wavelength_dim = 'wavelength', 
                                                                 suspect_threshold = 0.10, 
                                                                 fail_threshold = 0.30, 
                                                                 include_suspect_flags = False)
acs['blanket_flag_a_mts_proportional_gross_range'] = acsqaqc.blanket_gross_range_test(acs.flag_a_mts_proportional_gross_range, 
                                                                         wavelength_dim = 'wavelength', 
                                                                         suspect_threshold = 0.10, 
                                                                         fail_threshold = 0.30, 
                                                                         include_suspect_flags = False)

In [ ]:
acs = acs.where(acs.flag_elapsed_time != 4, drop = True)  # Remove samples with a flag of 4 for the elapsed time test
acs = acs.where(acs.flag_internal_temperature != 4, drop = True)  # Remove samples with a flag of 4 for the internal temperature test.
acs = acs.where(acs.flag_a_uncorrected_inf_nan != 4, drop = True)  # Remove samples with a flag of 4 for the absorption inf nan test.
acs = acs.where(acs.flag_c_uncorrected_inf_nan != 4, drop = True)  # Remove samples with a flag of 4 for the absorption inf nan test.
acs = acs.where(acs.blanket_flag_a_mts_proportional_gross_range != 4, drop = True)  # Remove samples with a flag of 4 for the absorption gross range test.
acs = acs.where(acs.blanket_flag_c_mts_gross_range != 4, drop = True)  # Remove samples with a flag of 4 for the absorption gross range test.

In [ ]:
acs = acs[['c_mts','a_mts_proportional', 'sea_water_conservative_temperature','sea_water_absolute_salinity','seawater_state','longitude','latitude',]]  

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6), constrained_layout=True)
p = ax.scatter(acs.longitude, acs.latitude, c = acs.seawater_state,cmap = 'bwr')
cbar = fig.colorbar(p, ax = ax, ticks = [0,1])
cbar.ax.set_yticklabels(['pg', 'g'])

ax.set_box_aspect((acs.latitude.max() - acs.latitude.min())/(acs.longitude.max() - acs.longitude.min()))

ax.yaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

In [ ]:
g = acs.where(acs.seawater_state != 0, drop = True)

In [ ]:
pg = acs.where(acs.seawater_state != 1, drop = True)

In [ ]:
def cleanup_filtered_periods(g, remove_start = 5 * 60, remove_end = 1 * 60):
    dt = g.where(g['time'].diff('time') > np.timedelta64(30 * 60, 's'), drop=True).get_index('time')
    fperiods = []
    jback = np.timedelta64(60, 's')  # 30 second jump back to avoid collecting data from the following profile
    for i, d in enumerate(dt):
        # pull out the profile
        if i == 0:
            fperiod = g.sel(time=slice(g['time'].values[0], d - jback))
        else:
            fperiod = g.sel(time=slice(dt[i - 1], d - jback))
            
            
        fperiod = fperiod.sel(time = slice(fperiod.time.min() + np.timedelta64(remove_start,'s'), 
                                           fperiod.time.max() - np.timedelta64(remove_end,'s')))    
        
        if len(fperiod.time.values != 0):
            fperiods.append(fperiod)
    
        # grab the last profile and append it to the list
        fperiod = g.sel(time=slice(d, g['time'].values[-1]))
        fperiod = fperiod.sel(time = slice(fperiod.time.min() + np.timedelta64(remove_start,'s'), 
                                           fperiod.time.max() - np.timedelta64(remove_end,'s')))    

        if len(fperiod.time.values != 0):
            fperiods.append(fperiod)
    
    filtered = xr.concat(fperiods, dim = 'time')
    filtered = filtered.drop_duplicates(dim = 'time')
    return filtered
    

In [ ]:
gc = cleanup_filtered_periods(g)
gi = gc.interp(time = pg.time, method = 'linear')

In [ ]:
p = pg - gi

In [ ]:
for dv in g.data_vars: 
    if 'mts' in dv:
        _dv = dv.replace('mts', 'g')
        g = g.rename({dv: _dv})

In [ ]:
for dv in p.data_vars: 
    if 'mts' in dv:
        _dv = dv.replace('mts', 'p')
        p = p.rename({dv: _dv})

In [ ]:
t = p.time.values[250000]

sp = p.sel(time = t)
sg = gi.sel(time = t)

fig, ax = plt.subplots(2,1, figsize = (12,5), constrained_layout = True)

ax[0].plot(sp.wavelength, sp.c_p)
ax[1].plot(sg.wavelength, sg.c_g)